# Import

In [1]:
%install-swiftpm-flags -c release
%install '.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.3")' SwiftCoreMLTools
%install '.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")' Just

Installing packages:
	.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.3")
		SwiftCoreMLTools
	.package(url: "https://github.com/dduan/Just.git", from: "0.8.0")
		Just
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmp61sxotst/swift-install
Fetching https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Fetching https://github.com/dduan/Just.git
Fetching https://github.com/apple/swift-protobuf.git
Cloning https://github.com/apple/swift-protobuf.git
Resolving https://github.com/apple/swift-protobuf.git at 1.8.0
Cloning https://github.com/dduan/Just.git
Resolving https://github.com/dduan/Just.git at 0.8.0
Cloning https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Resolving https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git at 0.0.3
[1/3] Compiling Just Just.swift
[2/3] Compiling SwiftProtobuf AnyMessageStorage.swift
[3/4] Compiling SwiftCoreMLTools Activations.swift
[4/5] Compiling jupyterInstalledPackages jupyterInstalled

In [2]:
import Foundation
import TensorFlow
import SwiftCoreMLTools
import Just

# Data Download

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
        - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - LSTAT    % lower status of the population
        - MEDV     Median value of owner-occupied homes in $1000's

    :Missing Attribute Values: None

    :Creator: Harrison, D. and Rubinfeld, D.L.

This is a copy of UCI ML housing dataset.
https://archive.ics.uci.edu/ml/machine-learning-databases/housing/


In [3]:
// !wget -O "../data/housing.csv" https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data

if let cts = Just.get(URL(string: "https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data")!).content {
    try! cts.write(to: URL(fileURLWithPath:"../data/housing.csv"))
}

# Data Ingestion

In [4]:
// TODO: Download housing.csv

let data = try String(contentsOfFile:"../data/housing.csv", encoding: String.Encoding.utf8)
let dataRecords: [[Float]] = data.split(separator: "\n").map{ String($0).split(separator: " ").compactMap{ Float(String($0)) } }

let numRecords = dataRecords.count
let numColumns = dataRecords[0].count

let dataFeatures = dataRecords.map{ Array($0[0..<numColumns-1]) }
let dataLabels = dataRecords.map{ Array($0[(numColumns-1)...]) }

let trainPercentage:Float = 0.8
let upToRecord = Int(ceil(Float(numRecords) * trainPercentage))

let trainFeatures = Array(dataFeatures[0..<upToRecord])
let trainLabels = Array(dataLabels[0..<upToRecord])
let testFeatures = Array(dataFeatures[upToRecord...])
let testLabels = Array(dataLabels[upToRecord...])

print("Training shapes [\(trainFeatures.count) \(trainFeatures[0].count)] [\(trainLabels.count) \(trainLabels[0].count)]")
print("Testing shapes  [\(testFeatures.count) \(testFeatures[0].count)] [\(testLabels.count) \(testLabels[0].count)]")

let xAllTrain = Array(trainFeatures.joined())
let yAllTrain = Array(trainLabels.joined())
let xAllTest = Array(testFeatures.joined())
let yAllTest = Array(testLabels.joined())

let XTrainDeNorm = Tensor<Float>(xAllTrain).reshaped(to: TensorShape([upToRecord, numColumns-1]))
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([upToRecord, 1]))
let XTestDeNorm = Tensor<Float>(xAllTest).reshaped(to: TensorShape([numRecords-upToRecord, numColumns-1]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numRecords-upToRecord, 1]))

Training shapes [405 13] [405 1]
Testing shapes  [101 13] [101 1]


# Normalize

In [5]:
let mean = XTrainDeNorm.mean(alongAxes: 0)
let std = XTrainDeNorm.standardDeviation(alongAxes: 0)

print(mean, std)

[[  2.0137098,   14.197531,    9.523555, 0.086419754,  0.53213036,   6.3311296,    64.47929,
    4.1678762,    6.832099,   353.68396,    18.03163,   379.84735,   11.394517]] [[ 6.5076075,  25.258776,   6.534038, 0.28098345, 0.11449408,  0.7311985,  29.000755,  2.1797554,
    6.544165,  132.14561,  2.2173452,  40.494495,   6.852825]]


In [6]:
let XTrain = (XTrainDeNorm - mean)/std
let XTest = (XTestDeNorm - mean)/std

In [7]:
print(XTrain.shape, YTrain.shape, XTest.shape, YTest.shape)

[405, 13] [405, 1] [101, 13] [101, 1]


# Model

In [8]:
struct RegressionModel: Layer {
    var layer1 = Dense<Float>(inputSize: 13, outputSize: 64, activation: relu)
    var layer2 = Dense<Float>(inputSize: 64, outputSize: 32, activation: relu)
    var layer3 = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        return input.sequenced(through: layer1, layer2, layer3)
    }
}

var model = RegressionModel()

# Training

In [9]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [10]:
let epochCount = 500
let batchSize = 32
let numberOfBatch = Int(ceil(Double(upToRecord) / Double(batchSize)))
let shuffle = true

func mae(predictions: Tensor<Float>, truths: Tensor<Float>) -> Float {
    return abs(Tensor<Float>(predictions - truths)).mean().scalarized()
}

In [11]:
for epoch in 1...epochCount {
    var epochLoss: Float = 0
    var epochMAE: Float = 0
    var batchCount: Int = 0
    var batchArray = Array(repeating: false, count: numberOfBatch)
    for batch in 0..<numberOfBatch {
        var r = batch
        if shuffle {
            while true {
                r = Int.random(in: 0..<numberOfBatch)
                if !batchArray[r] {
                    batchArray[r] = true
                    break
                }
            }
        }
        
        let batchStart = r * batchSize
        let batchEnd = min(upToRecord, batchStart + batchSize)
        let (loss, grad) = model.valueWithGradient { (model: RegressionModel) -> Tensor<Float> in
            let logits = model(XTrain[batchStart..<batchEnd])
            return meanSquaredError(predicted: logits, expected: YTrain[batchStart..<batchEnd])
        }
        optimizer.update(&model, along: grad)
        
        let logits = model(XTrain[batchStart..<batchEnd])
        epochMAE += mae(predictions: logits, truths: YTrain[batchStart..<batchEnd])
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochMAE /= Float(batchCount)
    epochLoss /= Float(batchCount)

    print("Epoch \(epoch): MSE: \(epochLoss), MAE: \(epochMAE)")
}

Epoch 1: MSE: 573.14496, MAE: 21.606884
Epoch 2: MSE: 519.6116, MAE: 20.265917
Epoch 3: MSE: 462.35956, MAE: 18.829098
Epoch 4: MSE: 398.8534, MAE: 17.20908
Epoch 5: MSE: 332.26373, MAE: 15.40366
Epoch 6: MSE: 267.21182, MAE: 13.475077
Epoch 7: MSE: 205.79198, MAE: 11.450288
Epoch 8: MSE: 151.43007, MAE: 9.416166
Epoch 9: MSE: 113.70744, MAE: 7.646505
Epoch 10: MSE: 88.67873, MAE: 6.600563
Epoch 11: MSE: 71.553604, MAE: 5.7828226
Epoch 12: MSE: 60.598816, MAE: 5.32742
Epoch 13: MSE: 51.04281, MAE: 4.812348
Epoch 14: MSE: 43.83846, MAE: 4.4834666
Epoch 15: MSE: 38.41464, MAE: 4.10994
Epoch 16: MSE: 34.59271, MAE: 3.9341304
Epoch 17: MSE: 31.08239, MAE: 3.696552
Epoch 18: MSE: 28.512743, MAE: 3.4807963
Epoch 19: MSE: 27.25426, MAE: 3.469846
Epoch 20: MSE: 25.701475, MAE: 3.3422394
Epoch 21: MSE: 24.475834, MAE: 3.2335176
Epoch 22: MSE: 23.634298, MAE: 3.133302
Epoch 23: MSE: 22.39583, MAE: 3.106403
Epoch 24: MSE: 21.406755, MAE: 2.983843
Epoch 25: MSE: 20.896555, MAE: 2.9506137
Epoch 26:

Epoch 204: MSE: 5.311083, MAE: 1.4937651
Epoch 205: MSE: 5.1837225, MAE: 1.4738076
Epoch 206: MSE: 4.7675557, MAE: 1.4460511
Epoch 207: MSE: 5.0894785, MAE: 1.4523365
Epoch 208: MSE: 5.032272, MAE: 1.4662381
Epoch 209: MSE: 5.2799306, MAE: 1.4810277
Epoch 210: MSE: 5.047757, MAE: 1.4519336
Epoch 211: MSE: 5.125017, MAE: 1.4625058
Epoch 212: MSE: 5.1180334, MAE: 1.4741045
Epoch 213: MSE: 5.10339, MAE: 1.4501736
Epoch 214: MSE: 5.0328207, MAE: 1.4764124
Epoch 215: MSE: 4.940192, MAE: 1.4509209
Epoch 216: MSE: 4.7850986, MAE: 1.446237
Epoch 217: MSE: 4.7451406, MAE: 1.4634526
Epoch 218: MSE: 5.164105, MAE: 1.4254438
Epoch 219: MSE: 5.0244927, MAE: 1.446285
Epoch 220: MSE: 4.9591265, MAE: 1.4363803
Epoch 221: MSE: 4.7817717, MAE: 1.420196
Epoch 222: MSE: 4.888365, MAE: 1.438548
Epoch 223: MSE: 4.92665, MAE: 1.441918
Epoch 224: MSE: 4.8920283, MAE: 1.4366777
Epoch 225: MSE: 4.8817835, MAE: 1.4321126
Epoch 226: MSE: 4.8229275, MAE: 1.4455292
Epoch 227: MSE: 4.7738376, MAE: 1.4285142
Epoch 22

Epoch 402: MSE: 2.928697, MAE: 1.115872
Epoch 403: MSE: 2.508293, MAE: 1.0839925
Epoch 404: MSE: 2.7186954, MAE: 1.1626478
Epoch 405: MSE: 2.853087, MAE: 1.1120098
Epoch 406: MSE: 2.6191943, MAE: 1.100196
Epoch 407: MSE: 2.599083, MAE: 1.0983578
Epoch 408: MSE: 2.7472272, MAE: 1.1571586
Epoch 409: MSE: 2.5136094, MAE: 1.0997775
Epoch 410: MSE: 2.545338, MAE: 1.0641929
Epoch 411: MSE: 2.654266, MAE: 1.1073043
Epoch 412: MSE: 2.6096654, MAE: 1.1160268
Epoch 413: MSE: 2.5696678, MAE: 1.1032634
Epoch 414: MSE: 2.626199, MAE: 1.1142333
Epoch 415: MSE: 2.5642552, MAE: 1.1176709
Epoch 416: MSE: 2.617486, MAE: 1.0769231
Epoch 417: MSE: 2.5213873, MAE: 1.1060184
Epoch 418: MSE: 2.3856032, MAE: 1.1031566
Epoch 419: MSE: 2.598826, MAE: 1.0743539
Epoch 420: MSE: 2.608653, MAE: 1.1231225
Epoch 421: MSE: 2.4721465, MAE: 1.0901202
Epoch 422: MSE: 2.5668116, MAE: 1.0739198
Epoch 423: MSE: 2.470141, MAE: 1.1121979
Epoch 424: MSE: 2.4688382, MAE: 1.0946416
Epoch 425: MSE: 2.458786, MAE: 1.0364736
Epoch 

# Test

In [12]:
Context.local.learningPhase = .inference

let cheat = model(XTrain)
print(cheat[0], YTrain[0])

let record: [Float] = [-0.4013274553772651,-0.48782210614046656,-1.1729760489283325,-0.2721895900613162,-0.8055945265354896,0.09156749405417394,-1.828902543802867,0.6384789935042571,-0.6351491942719604,0.1472680456555187,-0.7178137893787737,0.2073805740660824,-0.7473489168521552] 
let tfrecord = Tensor<Float>(record).reshaped(to: TensorShape([1, 13]))
let tfcheat = model(tfrecord)
print(tfcheat)

let prediction = model(XTest)
print(prediction[0], YTest[0])

let tmse = meanSquaredError(predicted: prediction, expected: YTest).scalarized()/Float(numRecords-upToRecord)
let tmae = mae(predictions: prediction, truths: YTest)/Float(numRecords-upToRecord)

print("MSE: \(tmse), MAE: \(tmae)")

[24.31423] [24.0]
[[23.353426]]
[12.247286] [5.0]
MSE: 0.6932303, MAE: 0.06229373


# Export

In [13]:
print(model.layer1.weight.shape, model.layer2.weight.shape, model.layer3.weight.shape)
print(model.layer1.bias.shape, model.layer2.bias.shape, model.layer3.bias.shape)

[13, 64] [64, 32] [32, 1]
[64] [32] [1]


In [14]:
let coremlModel = Model(version: 4,
                        shortDescription: "Regression",
                        author: "Jacopo Mangiavacchi",
                        license: "MIT",
                        userDefined: ["SwiftCoremltoolsVersion" : "0.0.3"]) {
    Input(name: "input", shape: [13], featureType: .Double)
    Output(name: "output", shape: [1], featureType: .Double)
    NeuralNetwork {
        InnerProduct(name: "dense1",
                     input: ["input"],
                     output: ["outDense1"],
                     weights: model.layer1.weight.flattened().scalars,
                     bias: model.layer1.bias.flattened().scalars,
                     inputChannels: 13,
                     outputChannels: 64,
                     updatable: false)
        ReLu(name: "Relu1",
             input: ["outDense1"],
             output: ["outRelu1"])
        InnerProduct(name: "dense2",
                     input: ["outRelu1"],
                     output: ["outDense2"],
                     weights: model.layer2.weight.flattened().scalars,
                     bias: model.layer2.bias.flattened().scalars,
                     inputChannels: 64,
                     outputChannels: 32,
                     updatable: false)
        ReLu(name: "Relu2",
             input: ["outDense2"],
             output: ["outRelu2"])
        InnerProduct(name: "dense3",
                     input: ["outRelu2"],
                     output: ["output"],
                     weights: model.layer3.weight.flattened().scalars,
                     bias: model.layer3.bias.flattened().scalars,
                     inputChannels: 32,
                     outputChannels: 1,
                     updatable: false)
    }
}

In [15]:
let coreMLData = coremlModel.coreMLData
try! coreMLData!.write(to: URL(fileURLWithPath: "../model/s4tf_train_model.mlmodel"))

# Sample Record for testing Inferencing on CoreML

In [16]:
print(XTrain[0], YTrain[0])

[ -0.30846816,   0.15054052,   -1.1039964,  -0.30756173,   0.05126577,   0.33352128,
  0.024851447, -0.035726987,  -0.89119065,  -0.43651816,   -1.2319369,   0.42111015,
  -0.93603975] [24.0]
